In [1]:
from typing import Annotated

from langchain_deepseek import ChatDeepSeek
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import BaseMessage
from typing_extensions import TypedDict

from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from pydantic import BaseModel, field_validator, ValidationError
from dotenv import load_dotenv
from langgraph.checkpoint.memory import MemorySaver

load_dotenv()

memory = MemorySaver()

class PydanticState(BaseModel):
    messages: Annotated[list, add_messages]

    # @field_validator('messages')
    # @classmethod
    # def validate_message(cls, value):
    #     # Ensure the mood is either "happy" or "sad"
    #     if value not in ["happy", "sad"]:
    #         raise ValueError("Each mood must be either 'happy' or 'sad'")
    #     return value

# try:
#     state = PydanticState(name="John Doe", mood="mad")
# except ValidationError as e:
#     print("Validation Error:", e)

class State(TypedDict):
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(PydanticState)


tool = TavilySearchResults(max_results=2)
tools = [tool]
llm = ChatDeepSeek(
    model="deepseek-chat",
    temperature=0,
)
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=[tool])
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
# Any time a tool is called, we return to the chatbot to decide the next step
graph_builder.add_edge("tools", "chatbot")
graph_builder.set_entry_point("chatbot")
graph = graph_builder.compile(checkpointer=memory)

In [3]:
from langchain_core.messages import HumanMessage, SystemMessage

config = {"configurable": {"thread_id": "1"}}

messages = [SystemMessage(content="You are a sassy litle chatbot, a fiesty one.")]
graph.invoke({"messages": messages}, config)

{'messages': [SystemMessage(content='You are a sassy litle chatbot, a fiesty one.', additional_kwargs={}, response_metadata={}, id='4887b351-fd3e-4d9a-8d6a-3004f5a012db'),
  AIMessage(content="Oh, look who's here! Ready to chat with the sassiest little bot around? Spill the tea, darling—what's on your mind? 😏", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 151, 'total_tokens': 185, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 151}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_3d5141a69a_prod0225', 'id': '75767a0c-f384-49f2-85ab-a100b7fa7241', 'finish_reason': 'stop', 'logprobs': None}, id='run-19d9754a-dde8-4d4d-8b4a-d5bcbd55f5f5-0', usage_metadata={'input_tokens': 151, 'output_tokens': 34, 'total_tokens': 185, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})]}

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

def stream_graph_updates(user_input: str):
    for event in graph.stream(
        {
            # "messages": [{"role": "user", "content": user_input}],
            "messages": [HumanMessage(content=user_input)]
        },
        config,
        # stream_mode="values"
        ):
        for value in event.values():
            print("🤖 AI:", value["messages"][-1].content)

messages = [SystemMessage(content="You are a sassy litle chatbot, a fiesty one.")]

while True:
    try:
        config = {"configurable": {"thread_id": "1"}}
        user_input = input("User: ")
        
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break

        stream_graph_updates(user_input)
    except:
        # fallback if input() is not available
        user_input = "What do you know about LangGraph?"
        print("🖋️ User: " + user_input)
        stream_graph_updates(user_input)
        break